In [13]:
import rasterio as rio
import rasterio.mask
from rasterio.warp import calculate_default_transform,reproject,Resampling
from shapely import geometry
import geopandas as gpd
from pyproj import Transformer, CRS

In [2]:
fp_uas = r"E:\USGS\Sonoma_fires\GLA3\GLA3_26910_rect_2021_interp_clip_test3.tif"
fp_sent2 = r'E:\USGS\Sent2\SENT2_L2A_04082021_rgb_32610.tiff'


In [22]:
with rio.open(fp_uas) as src:
    #profile = src.profile
    bb = [round(num,0) for num in src.bounds]

# transform from 26910 to 32610 for Sent 2 clip 

#transformer = Transformer.from_crs("epsg:26910", "epsg:32610")
#minx,miny = transformer.transform(new_bb[0],new_bb[1])
#maxx,maxy = transformer.transform(new_bb[2],new_bb[3])

#footprint = geometry.Polygon([[minx,maxy],[maxx,maxy],[maxx,miny],[minx,miny]])

footprint = geometry.Polygon([[bb[0],bb[3]],[bb[2],bb[3]],[bb[2],bb[1]],[bb[0],bb[1]]])

geo = gpd.GeoDataFrame({'geometry':footprint},index=[0],crs="epsg:26910")

In [14]:
crs = CRS.from_user_input(26910)

In [15]:
###### need to perform this section in memory


#dst_crs = {'init':"EPSG:26910"}  #UAS crs
dst_crs = crs
outfile = fp_sent2.replace('32610.tiff','26910.tiff')

with rasterio.open(fp_sent2) as src:
    src_transform = src.transform

    # calculate the transform matrix for the output
    dst_transform, width, height = calculate_default_transform(
        src.crs,
        dst_crs,
        src.width,
        src.height,
        *src.bounds,  # unpacks outer boundaries (left, bottom, right, top)
    )

    # set properties for output
    dst_kwargs = src.meta.copy()
    dst_kwargs.update(
        {
            "crs": dst_crs,
            "transform": dst_transform,
            "width": width,
            "height": height,
            "nodata": 0,  # replace 0 with np.nan
        }
    )

    with rasterio.open(outfile, "w", **dst_kwargs) as dst:
        # iterate through bands
        for i in range(1, src.count + 1):
            reproject(
                source=rasterio.band(src, i),
                destination=rasterio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=dst_transform,
                dst_crs=dst_crs,
                resampling=Resampling.nearest,
            )

In [23]:

outfile2 = outfile.replace('.tif','clipped.tif')

with rio.open(outfile) as src:
    out_image, out_transform = rasterio.mask.mask(src, geo.geometry, crop=True)
    out_meta = src.meta.copy()

out_meta.update({
    "driver": "GTiff",
    "height": out_image.shape[1],
    "width": out_image.shape[2],
    "transform": out_transform
})

with rio.open(outfile2, "w", **out_meta) as dest:
    dest.write(out_image)